<a href="https://colab.research.google.com/github/markusgoeswein/01_git/blob/master/HAM10000_skin_cancer_resnet50.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

   # Esempio di classificazione delle immagini
   
   Elaborato a partire dal tutorial di Hugging Face:
   https://colab.research.google.com/github/huggingface/notebooks/blob/main/examples/image_classification.ipynb
   
   Download dati da Kaggle a Colab:

In [1]:
! pip install -q kaggle

 Upload del file kaggle.json su Colab:

In [2]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"markusgswein","key":"684be3d66401616c0bba2b3d1e393c55"}'}

In [3]:
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

In [4]:
!kaggle datasets download -d kmader/skin-cancer-mnist-ham10000

100% 5.20G/5.20G [04:24<00:00, 23.8MB/s]
100% 5.20G/5.20G [04:24<00:00, 21.1MB/s]


In [ ]:
!unzip skin-cancer-mnist-ham10000.zip

In [ ]:
import pandas as pd

df = pd.read_csv('HAM10000_metadata.csv')

print(df.to_string())

In [7]:
lesion_type_dict = {
    'nv': 'Melanocytic-nevi',
    'mel': 'Melanoma',
    'bkl': 'Benign-keratosis-like-lesions',
    'bcc': 'Basal-cell-carcinoma',
    'akiec': 'Actinic-keratoses',
    'vasc': 'Vascular-lesions',
    'df': 'Dermatofibroma'
}

disease =[]
for i in range(len(df)):
    disease.append(lesion_type_dict[df.loc[i,"dx"]])

df['disease']=disease

In [8]:
import os
from pathlib import Path
data_dir = Path('images')
os.mkdir(data_dir)

for dis in list(lesion_type_dict.values()):
    path = str(data_dir) + "/" +dis
    os.mkdir(path)

In [9]:
import shutil

for i in range(len(df)):
    f= Path('HAM10000_images_part_1/' + df.loc[i,"image_id"] +'.jpg')
    if os.path.isfile(f):
        shutil.copy(f, Path('images/' + df.loc[i,"disease"]))
    else:
        shutil.copy(Path('HAM10000_images_part_2/' + df.loc[i,"image_id"] +'.jpg'), Path('images/' + df.loc[i,"disease"]))
    if i%1000==0:
        print(i)

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000


Si installano le librerie di Hugging Face:

In [10]:
!pip install -q datasets transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.3/519.3 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 99.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 21.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 26.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 105.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 68.9 MB/s eta 0:00:00


Ci si collega con Hugging Face dove bisogna creare e copiare un token necessario per il login:

In [11]:
from huggingface_hub import login
login()

In [12]:
from transformers import AutoImageProcessor
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.backends.cudnn as cudnn
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
from PIL import Image
from tempfile import TemporaryDirectory

cudnn.benchmark = True
plt.ion()   # interactive mode

model_checkpoint = "microsoft/resnet-50" # pre-trained model from which to fine-tune
batch_size = 32 # batch size for training and evaluation

image_processor  = AutoImageProcessor.from_pretrained(model_checkpoint)
image_processor

Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration.


ConvNextImageProcessor {
  "crop_pct": 0.875,
  "do_normalize": true,
  "do_rescale": true,
  "do_resize": true,
  "image_mean": [
    0.485,
    0.456,
    0.406
  ],
  "image_processor_type": "ConvNextImageProcessor",
  "image_std": [
    0.229,
    0.224,
    0.225
  ],
  "resample": 3,
  "rescale_factor": 0.00392156862745098,
  "size": {
    "shortest_edge": 224
  }
}

In [13]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [14]:
from torchvision.transforms import (
    CenterCrop,
    Compose,
    Normalize,
    RandomHorizontalFlip,
    RandomResizedCrop,
    Resize,
    ToTensor,
)

normalize = Normalize(mean=image_processor.image_mean, std=image_processor.image_std)
if "height" in image_processor.size:
    size = (image_processor.size["height"], image_processor.size["width"])
    crop_size = size
    max_size = None
elif "shortest_edge" in image_processor.size:
    size = image_processor.size["shortest_edge"]
    crop_size = (size, size)
    max_size = image_processor.size.get("longest_edge")

train_transforms = Compose(
        [
            RandomResizedCrop(crop_size),
            RandomHorizontalFlip(),
            ToTensor(),
            normalize,
        ]
    )

val_transforms = Compose(
        [
            Resize(size),
            CenterCrop(crop_size),
            ToTensor(),
            normalize,
        ]
    )



In [25]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision.models as models
from sklearn.model_selection import train_test_split

# Create an ImageFolder dataset
root_dir = 'images'

dataset = datasets.ImageFolder(root=root_dir)
class_names = dataset.classes
# Split dataset into training and testing sets
train_size = 0.8  # 80% for training, 20% for testing
train_indices, test_indices = train_test_split(list(range(len(dataset))), train_size=train_size, shuffle=True)


# Create DataLoaders for training and testing
batch_size = 32

train_dataset = torch.utils.data.Subset(dataset, train_indices)
train_dataset.dataset.transform = train_transforms
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)

test_dataset = torch.utils.data.Subset(dataset, test_indices)
test_dataset.dataset.transform = val_transforms
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=2)

dataloaders = {"train": train_dataloader, "val":test_dataloader}
dataset_sizes = {"train": len(train_dataset), "val":len(test_dataset)}
# Create DataLoaders for training and testing

# Loop through the batches of training data
for batch_images, batch_labels in train_dataloader:
    # Here you can use the batch of training images and labels for training your model
    # For simplicity, we'll just print the batch shapes
    print("Train batch images shape:", batch_images.shape)
    print("Train batch labels shape:", batch_labels.shape)
    break  # Let's just process the first batch for this example

# Loop through the batches of testing data
for batch_images, batch_labels in test_dataloader:
    # Here you can use the batch of testing images and labels for evaluating your model
    # For simplicity, we'll just print the batch shapes
    print("Test batch images shape:", batch_images.shape)
    print("Test batch labels shape:", batch_labels.shape)
    break  # Let's just process the first batch for this example

Train batch images shape: torch.Size([32, 3, 224, 224])
Train batch labels shape: torch.Size([32])
Test batch images shape: torch.Size([32, 3, 224, 224])
Test batch labels shape: torch.Size([32])


In [22]:
def train_model(model, criterion, optimizer, num_epochs=25):
    since = time.time()

    # Create a temporary directory to save training checkpoints
    with TemporaryDirectory() as tempdir:
        best_model_params_path = os.path.join(tempdir, 'best_model_params.pt')

        torch.save(model.state_dict(), best_model_params_path)
        best_acc = 0.0

        for epoch in range(num_epochs):
            print(f'Epoch {epoch}/{num_epochs - 1}')
            print('-' * 10)

            # Each epoch has a training and validation phase
            for phase in ['train', 'val']:
                if phase == 'train':
                    model.train()  # Set model to training mode
                else:
                    model.eval()   # Set model to evaluate mode

                running_loss = 0.0
                running_corrects = 0

                # Iterate over data.
                for inputs, labels in dataloaders[phase]:
                    inputs = inputs.to(device)
                    labels = labels.to(device)

                    # zero the parameter gradients
                    optimizer.zero_grad()

                    # forward
                    # track history if only in train
                    with torch.set_grad_enabled(phase == 'train'):
                        outputs = model(inputs)
                        _, preds = torch.max(outputs, 1)
                        loss = criterion(outputs, labels)

                        # backward + optimize only if in training phase
                        if phase == 'train':
                            loss.backward()
                            optimizer.step()

                    # statistics
                    running_loss += loss.item() * inputs.size(0)
                    running_corrects += torch.sum(preds == labels.data)
                # if phase == 'train':
                #     scheduler.step()

                epoch_loss = running_loss / dataset_sizes[phase]
                epoch_acc = running_corrects.double() / dataset_sizes[phase]

                print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

                # deep copy the model
                if phase == 'val' and epoch_acc > best_acc:
                    best_acc = epoch_acc
                    torch.save(model.state_dict(), best_model_params_path)

            print()

        time_elapsed = time.time() - since
        print(f'Training complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
        print(f'Best val Acc: {best_acc:4f}')

        # load best model weights
        model.load_state_dict(torch.load(best_model_params_path))
    return model

In [26]:

model = models.resnet50(weights= models.ResNet50_Weights.IMAGENET1K_V2)
num_features = model.fc.in_features
model.fc = nn.Linear(num_features, 7)
model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Fine-tuning loop
num_epochs = 10

train_model(model = model, criterion = criterion, optimizer = optimizer, num_epochs=10)



Epoch 0/9
----------
train Loss: 0.7812 Acc: 0.7234
val Loss: 0.7799 Acc: 0.7239

Epoch 1/9
----------
train Loss: 0.6146 Acc: 0.7767
val Loss: 0.5636 Acc: 0.7883

Epoch 2/9
----------
train Loss: 0.5197 Acc: 0.8138
val Loss: 0.8047 Acc: 0.7124

Epoch 3/9
----------
train Loss: 0.4584 Acc: 0.8333
val Loss: 0.5891 Acc: 0.7808

Epoch 4/9
----------
train Loss: 0.3828 Acc: 0.8598
val Loss: 0.6747 Acc: 0.7798

Epoch 5/9
----------
train Loss: 0.3115 Acc: 0.8849
val Loss: 0.7476 Acc: 0.7893

Epoch 6/9
----------
train Loss: 0.2533 Acc: 0.9056
val Loss: 0.5855 Acc: 0.8183

Epoch 7/9
----------
train Loss: 0.1992 Acc: 0.9291
val Loss: 0.9308 Acc: 0.7339

Epoch 8/9
----------
train Loss: 0.1594 Acc: 0.9456
val Loss: 0.7144 Acc: 0.7968

Epoch 9/9
----------
train Loss: 0.1127 Acc: 0.9621
val Loss: 0.7137 Acc: 0.7993

Training complete in 19m 51s
Best val Acc: 0.818273


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [31]:
torch.save(model.state_dict(), 'HAM10000_skincancer_resnet50.pth')

In [34]:
softmax_outputs_list = []
model.eval()

with torch.no_grad():
  for inputs, labels in test_dataloader:
    inputs = inputs.to(device)
    labels = labels.to(device)

    outputs = model(inputs)
    softmax_outputs = torch.nn.functional.softmax(outputs, dim = 1)
    softmax_outputs_list.append(softmax_outputs)

all_softmax_outputs = torch.cat(softmax_outputs_list, dim = 0)


In [52]:
# all_softmax_outputs
all_softmax_outputs = all_softmax_outputs.cpu()


array([[4.6632960e-01, 5.0432660e-05, 2.5098981e-02, ..., 2.5828308e-03,
        5.0509822e-01, 9.9509151e-08],
       [1.1885130e-09, 1.5078506e-08, 3.7272446e-07, ..., 9.9996841e-01,
        5.5827386e-06, 2.4520752e-05],
       [4.4951750e-07, 4.0993094e-11, 3.5951351e-07, ..., 9.9993539e-01,
        6.3399624e-05, 2.7904468e-07],
       ...,
       [1.9547178e-05, 1.9846593e-07, 3.3851969e-04, ..., 9.0865171e-01,
        9.0974636e-02, 3.4055572e-06],
       [8.2908527e-06, 9.4750000e-08, 9.3873491e-04, ..., 9.9821991e-01,
        8.1932230e-04, 2.7426461e-06],
       [6.7092247e-07, 4.0771392e-05, 1.9016134e-04, ..., 9.8250681e-01,
        5.9949444e-03, 1.1257494e-02]], dtype=float32)

In [57]:
np.save("softmax_outputs.npy", all_softmax_outputs)

In [62]:
np.save("test_indices", np.array(test_indices))

In [67]:
val_labels = [x[1] for x in test_dataset]

In [69]:
np.save("val_labels", np.array(val_labels))

In [78]:
classes = {i+1:x for i,x in enumerate(class_names)}

In [82]:
pickle_out = open("dict_int_to_label","wb")
pickle.dump(classes,pickle_out)